#### Importing Library 

In [1]:
%%time

from nltk.corpus import stopwords 
from nltk import download 
download('stopwords')

Wall time: 13.2 s


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JPG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
stop_words = stopwords.words('english')

In [3]:
from pyemd import emd
import gensim
from gensim.models import Word2Vec


In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format('D:/GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [5]:
model.init_sims(replace=True)

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity

import torch

In [7]:
%%time
embed = hub.load("D:/USE")

Wall time: 31.5 s


In [8]:
def cos_sim(input_vectors):
    similarity = cosine_similarity(input_vectors)
    return similarity

In [12]:
negative = ["not" , "without","against","bad","useless","no","dislike","hate"]

def semantic_similarity(actual_answer , given_answer) :
    actual = actual_answer.lower().split(".")
    given = given_answer.lower().split(".")
    
    
    
    sim_checker = actual 
    
    not_matching_semantics = list()
    
    semantic_1 = 0   # Actual_answer
    semantic_2 = 0   # Given_answee
    
    actual_embed_list = list()
    given_embed_list = list()
    
    
    
    for z in range(len(actual)) :
        list_actual = list()  
        list_actual.append(actual[z])
        actual_embed_list.append(embed(list_actual))
        #print(actual_embed_list[z].shape)
    
    for z in range(len(given)) :
        
        semantic_1 = 0
        semantic_2 = 0 
        list_given = list()
        list_given.append(given[z])
        embed_z = embed(list_given)
        
        
        sim_check = sim_checker.copy() 
        sim_check.append(given[z]) 
        
        sen_em = embed(sim_check)
        
        similarity_matrix = cos_sim(np.array(sen_em))
        
        similarity_matrix_df = pd.DataFrame(similarity_matrix) 
        
        cos_list = list(similarity_matrix_df[len(similarity_matrix_df) - 1]) 
        cos_list = cos_list[:len(cos_list)-1]
        #print(cos_list)
        
        index = cos_list.index(max(cos_list))
        
        actual_check = actual[index]
        actual_check = actual_check.split()
        for i in range(len(actual_check) - 1) :
            if(actual_check[i] in negative and actual_check[i+1] in negative) :
                semantic_1 += 1 
            elif(actual_check[i] in negative and actual_check[i+1] not in negative) :
                semantic_1 -= 1 
                
        
        


        answer_given = given[z].split()
        for i in range(len(answer_given) - 1) :
            if(answer_given[i] in negative and answer_given[i+1] in negative) :
                semantic_2 += 1 
            elif(answer_given[i] in negative and answer_given[i+1] not in negative) :
                semantic_2 -= 1 

                
        
        if(semantic_1 == 0 and semantic_2 == 0) :
            
            """
            Well and good
            """
        elif(semantic_1 < 0  and semantic_2 >= 0) :
            not_matching_semantics.append(list([actual[index],given[z]]))
            embed_z*=(-1)
            
        
        elif(semantic_1 >= 0 and semantic_2 < 0 ) :
            not_matching_semantics.append(list([actual[index],given[z]]))
            embed_z*=(-1)
            
        
        
        #print(semantic_1,semantic_2,actual[index],given[z])
        
        
        given_embed_list.append(embed_z)
        
        
        
    
    #print(np.array(actual_embed_list).shape)
    actual_embed = actual_embed_list[0] 
    #print(actual_embed.shape) 
    
    for i in range(len(actual_embed_list)-1) :
        #print(actual_embed_list[i+1].shape)
        actual_embed += actual_embed_list[i+1]
        
    given_embed = given_embed_list[0] 
    for i in range(len(given_embed_list) - 1) :
        given_embed += given_embed_list[i+1] 
        
        
    
            
    actual_embed = np.array(actual_embed).reshape(512)
    given_embed = np.array(given_embed).reshape(512) 
    sem_checker = list([actual_embed,given_embed]) 
    answer = pd.DataFrame(cos_sim(sem_checker))
            
        
    return not_matching_semantics , answer[0][1]

In [13]:
a = "Jay is a undergrad student . He is studying CSE and is not interested in ED . He is from VNIT . He loves Ice cream"  
b = "Jay is a postgrad engineer .  He is doing his PG in CSE and ED . He is not from VNIT . He do not hate ice cream" 

In [14]:
not_matching , similarity = semantic_similarity(a,b)

In [15]:
def WMD(actual_answer , given_answer,model) :
    
    actual_answer = actual_answer.lower().split()
    actual_answer = [w for w in actual_answer if w not in stop_words]
    
    
    given_answer = given_answer.lower().split()
    given_answer = [w for w in given_answer if w not in stop_words]
    
    
    return model.wmdistance(given_answer,actual_answer)
    
    

In [16]:
distance = WMD(a,b,model)

In [17]:
distance

0.5904527414524166

In [18]:
def score(given_answer , actual_answer,model) :
    
    given_answer1 = given_answer
    actual_answer1 = actual_answer
    
    given_answer2 = given_answer
    actual_answer2 = actual_answer

    not_macthing , similarity = semantic_similarity(actual_answer1,given_answer1)
    distance = WMD(actual_answer2,given_answer2,model)
    

    
    if(similarity > 0) :
        if(distance == 0) :
            return 1 
        return similarity/distance
    else :
        return -1 

In [19]:
score(a,b,model)

-1

In [20]:
import wikipedia 

In [21]:
page1 = wikipedia.page("Republic of India")
a = page1.summary

In [22]:
page2 = wikipedia.page("Demographics of India")
b = page2.summary

In [23]:
page3 = wikipedia.page("Demographics of United States")
c = page3.summary

In [ ]:
len(a) , len(b), len(c)

In [24]:
score(a,b,model)

0.5976578001860909

In [25]:
score(a,c,model)

0.2617015521585175

In [26]:
score(b,c,model)

0.6956970576797735